In [1]:
import json
import pandas as pd
import re
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

from keras.preprocessing import sequence
from keras.utils import np_utils

import os
import shutil
import urllib3
import zipfile
import pandas as pd
import json
import ast

In [ ]:
path='/Users/AlexYoung/Downloads/proj/proj_3/[라벨]경상도_학습데이터_1'
file_list=os.listdir(path)
file_list_py=[file for file in file_list if file.endswith('.json')]
file_list_py.sort()
#print(file_list_py)
data=file_list_py[0:100]
print(data)


In [90]:
f = open("[라벨]경상도_학습데이터_1/DKCI20000001.json", encoding="UTF-8")
raw_data = json.loads(f.read())

In [93]:
type(raw_data["utterance"])

list

In [82]:
a=[{1:1,2:2,3:3},{4:4,5:5}]
b=[{1:11,2:22,3:33},{4:44,5:55}]
a.extend(b)
a

[{1: 1, 2: 2, 3: 3}, {4: 4, 5: 5}, {1: 11, 2: 22, 3: 33}, {4: 44, 5: 55}]

In [2]:
raw_data={"utterance":[]}
for k in range(0,100):
    name="DKSR200000"+str(k)+".json"
    try:
        f = open("[라벨]경상도_학습데이터_1/"+name, encoding="UTF-8")
        data = json.loads(f.read())
        raw_data["utterance"].extend(data["utterance"])
    except:
        pass

    

In [98]:
raw_data["utterance"][500]

{'id': 'DKSR20000011.1.1.281',
 'form': '그런 데가 재밌고 나머지는 다 비슷비슷한 거 같애.',
 'standard_form': '그런 데가 재밌고 나머지는 다 비슷비슷한 거 같애.',
 'dialect_form': '그런 데가 재밌고 나머지는 다 비슷비슷한 거 같애.',
 'speaker_id': 'SR0062',
 'start': 1324.68,
 'end': 1330.89,
 'note': '',
 'eojeolList': [{'id': 1,
   'eojeol': '그런',
   'standard': '그런',
   'isDialect': False},
  {'id': 2, 'eojeol': '데가', 'standard': '데가', 'isDialect': False},
  {'id': 3, 'eojeol': '재밌고', 'standard': '재밌고', 'isDialect': False},
  {'id': 4, 'eojeol': '나머지는', 'standard': '나머지는', 'isDialect': False},
  {'id': 5, 'eojeol': '다', 'standard': '다', 'isDialect': False},
  {'id': 6, 'eojeol': '비슷비슷한', 'standard': '비슷비슷한', 'isDialect': False},
  {'id': 7, 'eojeol': '거', 'standard': '거', 'isDialect': False},
  {'id': 8, 'eojeol': '같애.', 'standard': '같애.', 'isDialect': False}]}

In [99]:
type(raw_data["utterance"])

list

In [4]:
# 사투리, 표준어 사전 생성 함수
def make_dict(raw_data):

    dialect_word = []
    standard_word = [] 
    all_std_word=[]

    filter = re.compile('[^가-힣 +]')    
    
    for i in range(len(raw_data["utterance"])):
        #print(raw_data["utterance"])
        eojeol = raw_data["utterance"][i]["eojeolList"]
            
        for k in range(len(eojeol)):
            if eojeol[k]["isDialect"] == True:
                if k not in dialect_word:
                    dlt = filter.sub('', eojeol[k]['eojeol'])
                    std = filter.sub('', eojeol[k]['standard'])
                    dialect_word.append(dlt)
                    standard_word.append(std)
            else:
                if k not in standard_word:
                    word = filter.sub('', eojeol[k]['standard'])
                    all_std_word.append(word)

    word_dictionary = pd.DataFrame([dialect_word, standard_word])
    word_dictionary = word_dictionary.transpose()
    word_dictionary.columns = ['dialect', 'standard']
    dialect_dictionary = word_dictionary['dialect']
    standard_dictionary = word_dictionary['standard']
    dialect_dict = dialect_dictionary.to_dict()
    standard_dict = standard_dictionary.to_dict()

    index_to_dialect = {v:k for k,v in dialect_dict.items()}
    index_to_standard = {v:k for k,v in standard_dict.items()}

    dialect_to_index = {v:k for k,v in index_to_dialect.items()}
    standard_to_index = {v:k for k,v in index_to_standard.items()}    

    return dialect_to_index,standard_to_index,index_to_dialect,index_to_standard,all_std_word
   

In [5]:
index_to_dialect,index_to_standard,dialect_to_index,standard_to_index, all_std_word = make_dict(raw_data)

In [6]:
# 사투리 표준어 쌍 생성 함수
def make_pair(raw_data):
    filter = re.compile('[^가-힣+]')

    dialect_form = []
    standard_form = []

    for i in range(len(raw_data["utterance"])):
        sentence = raw_data["utterance"][i]
        
        for j in range(len(sentence["eojeolList"])):
            #print(sentence["eojeolList"][0]['eojeol'])
            if sentence["eojeolList"][j]['isDialect'] == True:
                try : 

                    dialect = sentence["eojeolList"][j]["eojeol"]
                    standard = sentence["eojeolList"][j]["standard"]
                    
                    dialect = filter.sub('', dialect)
                    standard = filter.sub('', standard)
                    
                    dialect = dialect_to_index[dialect]
                    standard = standard_to_index[standard]

                    dialect_form.append(dialect)
                    standard_form.append(standard)
                except:
                    pass

    result = pd.DataFrame(data = {"dialect_words":dialect_form, "standard_words": standard_form})


    return result
               

In [7]:
pairs = make_pair(raw_data)
pairs

,dialect_words,standard_words
0,920,920
1,793,793
2,2,920
3,920,920
4,1210,1210
...,...,...
1196,1210,1210
1197,1210,1210
1198,1210,1210
1199,1210,1210


In [8]:
# 사투리 사전에 존재하는지 확인하는 함수
def exist_in_dialect(x):

    if x in dialect_to_index:
        return True
    else:
        return False


In [9]:
trans_xtrain = np_utils.to_categorical(pairs["dialect_words"])
trans_ytrain = np_utils.to_categorical(pairs["standard_words"])
len(trans_xtrain[0])

1211

In [10]:
trans_xtrain.shape
trans_ytrain.shape


(1201, 1211)

In [14]:
translate_model = Sequential()
translate_model.add(Dense(256, input_dim = 1211, activation="relu"))
translate_model.add(Dense(1211, activation="softmax"))

In [15]:
translate_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
translate_model.fit(trans_xtrain, trans_ytrain, batch_size=100, epochs=100)

Epoch 1/100
13/13 [==============================] - 1s 16ms/step - loss: 7.0401 - accuracy: 0.3872
Epoch 2/100
13/13 [==============================] - 0s 20ms/step - loss: 6.8630 - accuracy: 0.5604
Epoch 3/100
13/13 [==============================] - 0s 17ms/step - loss: 6.5843 - accuracy: 0.5504
Epoch 4/100
13/13 [==============================] - 0s 17ms/step - loss: 6.1412 - accuracy: 0.5154
Epoch 5/100
13/13 [==============================] - 0s 19ms/step - loss: 5.4721 - accuracy: 0.4538
Epoch 6/100
13/13 [==============================] - 0s 20ms/step - loss: 4.5982 - accuracy: 0.4471
Epoch 7/100
13/13 [==============================] - 0s 18ms/step - loss: 3.7119 - accuracy: 0.4471
Epoch 8/100
13/13 [==============================] - 0s 23ms/step - loss: 3.1191 - accuracy: 0.4471
Epoch 9/100
13/13 [==============================] - 0s 18ms/step - loss: 2.7722 - accuracy: 0.4538
Epoch 10/100
13/13 [==============================] - 0s 23ms/step - loss: 2.5262 - accuracy: 0.4921

In [123]:
# 사투리 단어 번역 함수
def translate(word):
    index = dialect_to_index[word]
    one_hot = np_utils.to_categorical(index,len(trans_xtrain[0])) #인덱스 배열의크기 지정 필요 size_of_dialect
    print(translate_model.predict(np.array([one_hot])))
    pred_index = np.argmax(translate_model.predict(np.array([one_hot])))
    res = index_to_standard[pred_index]


    # for i in range(len(sentence)):
    #     if exist_in_dialect(sentence[i]) == True:
    #         print(dialect_to_index[sentence[i]])
    #         index = dialect_to_index[sentence[i]]
    #         one_hot = np_utils.to_categorical(index,len(xtrain[0])) #인덱스 배열의크기 지정 필요 size_of_dialect
    #         pred_index = np.argmax(translate_model.predict(np.array([one_hot])))
    #         res = res + " " + index_to_standard[pred_index]

    #     else:
    #         res = res + " " + sentence[i]

    return res


In [124]:
translate("했다")

[[2.4975409e-07 2.7200417e-07 2.4866893e-07 ... 2.2092448e-07
  2.5114656e-07 1.4238237e-04]]


'했지'

In [19]:
a_tag = ['NP','VP','AP','VNP','DP','IP','X','L','R']
b_tag = ['SBJ','OBJ','MOD','AJT','CMP','CNJ']
c_tag = []
pos_dict = {}
for i in a_tag:
    for j in b_tag:
        c_tag.append(i+"_"+j)
c_tag.extend(a_tag)
c_tag.extend(b_tag)
for i in range(len(c_tag)):
    pos_dict[c_tag[i]] = i

In [20]:
def make_eojeol(standard_to_index, dialect_to_index, all_std_word):

    eojeol_list=[]
    st_list = []
    di_list = []


    for k in standard_to_index:
        st_list.append(k)

    for v in dialect_to_index:
        di_list.append(v)     

    for i, j in zip(st_list, di_list):
        eojeol_list.append(i)
        eojeol_list.append(j + "0")
        
    for i in all_std_word:
        eojeol_list.append(i) 

    eojeol_df = pd.DataFrame(eojeol_list, columns=None)
    eojeol_dict = eojeol_df.to_dict()
    eojeol_dict = eojeol_dict[0]
    eojeol_dict = {v:k for k,v in eojeol_dict.items()}

    #print(eojeol_dict)

    return eojeol_dict

In [21]:
eojeol_dict = make_eojeol(standard_to_index, dialect_to_index, all_std_word)


In [22]:
# 
def get_mods_pos(std, dlt):

    filter = re.compile('[^가-힣 +]')

    dlt = filter.sub('', dlt)
    std = filter.sub('', std)
    
    
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU"

    accessKey = "7f039a3d-31ea-40ec-938e-4bbfbb0fbc16"
    analysisCode = "dparse"

    text = std
    
    requestJson = {
        "access_key": accessKey,
        "argument": {
            "text": text,
            "analysis_code": analysisCode
        }
    }

    http = urllib3.PoolManager()

    response = http.request(
        "POST",
        openApiURL,
        headers={"Content-Type": "application/json; charset=UTF-8"},
        body=json.dumps(requestJson)
    )

    tmp = ast.literal_eval(response.data.decode('utf-8'))
    dlt = dlt.split()
    std = std.split()
    words = []
    poses = []
    # print(std)
    #print(tmp["return_object"]["sentence"][0]["dependency"])
    try : 
        for i in range(len(tmp["return_object"]["sentence"][0]["dependency"])):
            dict_with_mods = tmp["return_object"]["sentence"][0]["dependency"][i]["mod"]
            #print(tmp["return_object"]["sentence"][0]["dependency"][i]["text"])
            for m in dict_with_mods:
                #print("std",std[i])
                if std[i] != dlt[i]:
                    words.append(eojeol_dict[dlt[i]+"0"])
                    poses.append(pos_dict[tmp["return_object"]["sentence"][0]["dependency"][int(m)]["label"]])

                    words.append(eojeol_dict[std[i]])
                    poses.append(pos_dict[tmp["return_object"]["sentence"][0]["dependency"][int(m)]["label"]])
                else:
         
                    words.append(eojeol_dict[std[i]])
                    #print(tmp["return_object"]["sentence"][0]["dependency"][int(m)]["label"])
                    poses.append(pos_dict[tmp["return_object"]["sentence"][0]["dependency"][int(m)]["label"]])
    except:
        return [],[]

    #print(words, poses)
    return words, poses


In [23]:
# 단어별 인덱스와 연결된 단어의 품사 인덱스 
def eojeol_dependence(raw_data):
    words=[]
    poses=[]

    for i in raw_data["utterance"]:

        std = i["standard_form"]
        dlt = i["dialect_form"]
        #print(i, std, dlt)
        word, pos = get_mods_pos(std, dlt)
        words.extend(word)
        poses.extend(pos)

    df = pd.DataFrame(data = {"pos":poses, "word":words })
        
    return df

In [24]:
train_df = eojeol_dependence(raw_data)

In [27]:
train_df[:10]

,pos,word
0,59,727
1,54,727
2,54,727
3,54,167747
4,1,146464
5,56,146464
6,3,146464
7,8,169402
8,54,136476
9,54,121467


In [48]:
max(train_df["word"])

169412

In [545]:
# 단어별 인덱스와 연결된 단어의 품사 인덱스
#def eojeol_dependence_():

    # 언어 분석 기술(문어)
    # openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU"
    # accessKey = "aba3a45d-3318-4061-8a5d-799c8521b082"
    # analysisCode = "dparse"

    # text = "엑소브레인은 내 몸 바깥에 있는 인공 두뇌라는 뜻으로, 세계 최고인공지능 기술 선도라는 비전을 달성하기 위한 과학기술정보통신부 소프트웨어 분야의 국가 혁신기술 개발형 연구개발 과제이다."
    # requestJson = {
    #     "access_key": accessKey,
    #     "argument": {
    #         "text": text,
    #         "analysis_code": analysisCode
    #     }
    # }
    # http = urllib3.PoolManager()
    # response = http.request(
    #     "POST",
    #     openApiURL,
    #     headers={"Content-Type": "application/json; charset=UTF-8"},
    #     body=json.dumps(requestJson)
    # )

    # s=ast.literal_eval(response.data.decode('utf-8'))

    # dep=s['return_object']['sentence'][0]['dependency']
    
    # eojeol_d=[]
    # pos_d=[]

    # re_eojeol_dict = {v:k for k,v in eojeol_dict.items()} #{인덱스:문자}
        
    # for i in range(len(dep)): #dep(딕셔너리)길이만큼 반복 23
    #     if dep[i]['text'] in eojeol_dict:
    #         eojeol_d.append(eojeol_dict.values())
    #     for j in range(len(dep[i]['mod'])):
    #         mod=dep[i]['mod']
    #         print(mod[j])
    #         if len(mod[j]) > 0:
    #             pos=re_eojeol_dict.get(mod[j])#결과값:문자
    #             #mod가 가리키는 어절의 품사를 확인하는 방법..
    #             if dep[i]['label'] in pos_dict:
    #                 pos_d.append(pos_dict.values())
    #             else:
    #                 return None
                
    # result = pd.DataFrame(data = {"word_dic":eojeol_d, "mod_dic":pos_d})
    # return result



In [26]:
synonym_xtrain = np_utils.to_categorical(train_df["pos"])
synonym_ytrain = np_utils.to_categorical(train_df["word"])
len(synonym_xtrain[0])


MemoryError: Unable to allocate 25.7 GiB for an array with shape (40793, 169413) and data type float32

In [547]:
synonym_xtrain.shape
synonym_ytrain.shape

(2894, 2989)

In [30]:
synonym_xtrain = np_utils.to_categorical(train_df["pos"])

In [31]:
synonym_xtrain.shape

(40793, 61)

In [62]:
result.shape

TensorShape([40793, 1000])

In [81]:
synonym_model = Sequential()
# synonym_model.add(Embedding(169412, 150, input_length=40793))
synonym_model.add(Dense(512, input_dim = 61, activation="relu"))
synonym_model.add(Dense(256,activation="relu"))
synonym_model.add(Dense(128,activation="relu"))
synonym_model.add(Dense(1000, activation="softmax"))

In [82]:
synonym_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [83]:
synonym_model.fit(synonym_xtrain, result, batch_size=500, epochs=100)

Epoch 1/100
82/82 [==============================] - 5s 39ms/step - loss: -4485.1309 - accuracy: 9.8056e-04
Epoch 2/100
82/82 [==============================] - 3s 41ms/step - loss: -508698.0312 - accuracy: 0.0013
Epoch 3/100
82/82 [==============================] - 3s 41ms/step - loss: -5983371.0000 - accuracy: 8.0896e-04
Epoch 4/100
82/82 [==============================] - 4s 45ms/step - loss: -28978180.0000 - accuracy: 0.0010
Epoch 5/100
82/82 [==============================] - 4s 44ms/step - loss: -90147984.0000 - accuracy: 0.0012
Epoch 6/100
82/82 [==============================] - 3s 42ms/step - loss: -215801792.0000 - accuracy: 8.8250e-04
Epoch 7/100
82/82 [==============================] - 4s 43ms/step - loss: -436835680.0000 - accuracy: 0.0015
Epoch 8/100
82/82 [==============================] - 3s 42ms/step - loss: -788206080.0000 - accuracy: 0.0012
Epoch 9/100
82/82 [==============================] - 4s 43ms/step - loss: -1307302656.0000 - accuracy: 0.0013
Epoch 10/100
82/82

In [ ]:
synonym_model.predict("")

37

In [118]:
# 동의어 처리를 포함한 사투리 번역 함수
def predict_synonym(sentence):
    global index_to_standard
    res = ""
    sentence_sep = sentence.split()
    #print(sentence_sep)
    for i in range(len(sentence_sep)):
        if exist_in_dialect(sentence_sep[i]) == True: #사투리 사전에 있으면
            if sentence_sep[i] in eojeol_dict.keys(): #해당 단어가 전체 사전에 있으면

                pos, _ = get_mods(sentence, i) #연관된 단어의 품사 받아옴
                pos_idx = pos_dict[pos] #품사의 인덱스
                word = sentence_sep[i] 
                word0_idx = eojeol_dict[word+"0"] #단어0의 인덱스
                word_idx = eojeol_dict[word] #단어의 인덱스
                # print(word_idx)
                # temp = result[word_idx]
                # print(type(temp))

                one_hot = np_utils.to_categorical(pos_idx,len(synonym_xtrain[0])) #인덱스 배열의크기 지정 필요 size_of_dialect
                print(type(one_hot))
                percentage = synonym_model.predict(np.array([one_hot])) # 동의어 처리 모델
                print(percentage[0])
                res_idx = max(percentage[word0_idx], percentage[word_idx]) # 둘 중 더 큰 확률값
                # print()
                one_hot = np_utils.to_categorical(res_idx,len(trans_xtrain[0])) #인덱스 배열의크기 지정 필요 size_of_dialect

                pred_index = np.argmax(translate_model.predict(np.array([one_hot]))) # 번역 모델

                
                # print("번역완료")
                # print("")

                res = res + " " + index_to_standard[pred_index]
            else: #해당 단어가 전체 사전에 없으면 -> 동의어가 없으므로 바로 번역 모델에 넣음
                
                word = sentence_sep[i]
                #print(word,"동의어가 없으므로 바로 번역 모델에 넣음")
                # word0_idx = eojeol_dict[word+"0"] #단어0의 인덱스
                # print(word0_idx)
                # one_hot = np_utils.to_categorical(word0_idx,len(trans_xtrain[0])) #인덱스 배열의크기 지정 필요 size_of_dialect
                # pred_index = np.argmax(translate_model.predict(np.array([one_hot])))
                res = res + " " + translate(word)
        else:
            word = sentence_sep[i]
            #print(word,"사투리가아님")
            res = res + " " + word
            
    
    return res

In [122]:
translate_model.predict("했다")

IndexError: list index out of range

In [119]:
predict_synonym("니처럼 어린 아이가 무슨 운전 면허증을 따노")

<class 'numpy.ndarray'>
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0.

IndexError: index 37 is out of bounds for axis 0 with size 1

In [95]:
# 단어와 연관된 단어의 품사 추출
def get_mods(sentence, num):
    #print(sentence, num)
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU"

    accessKey = "bd6d2bff-4e44-4035-8591-947fda262051"
    analysisCode = "dparse"

    text = sentence
    
    requestJson = {
        "access_key": accessKey,
        "argument": {
            "text": text,
            "analysis_code": analysisCode
        }
    }

    http = urllib3.PoolManager()

    response = http.request(
        "POST",
        openApiURL,
        headers={"Content-Type": "application/json; charset=UTF-8"},
        body=json.dumps(requestJson)
    )

    tmp = ast.literal_eval(response.data.decode('utf-8'))
    #print(tmp)
    dict_with_mods = tmp["return_object"]["sentence"][0]["dependency"]
    id_dict={}
    #print(dict_with_mods)
    for i in dict_with_mods[num]["mod"]:
        id_dict[dict_with_mods[int(i)]["weight"]] = int(i)
        #print(i)
    id = id_dict[max(id_dict)]
    
    pos = dict_with_mods[id]["label"]

    
    return pos, dict_with_mods[id]["text"]

In [90]:
predict_synonym("코로나 터지고 나서 배달을 한다")

코로나 터지고 나서 배달을 한다 4
{'request_id': '', 'result': -1, 'reason': 'Daily amount Limit has been exceed!:6049'}


KeyError: 'return_object'

In [91]:
translate("한다")

'하지'

In [653]:
# for i in eojeol_dict.keys():
#     for j in eojeol_dict.keys():
#         if i[-1]=="0":
#             tmp=i[:-1]
#             if tmp==j:
#                 print(tmp, j)

eojeol_dict["한다0"]

281

In [636]:
import os, requests, json, wave
from IPython.display import Audio
from IPython.display import display

url = "https://kakaoi-newtone-openapi.kakao.com/v1/synthesize"
key = '8d71d9fd039648d202453dcfaf984f88'

headers = {
    "Content-Type": "application/xml",
    "Authorization": "KakaoAK " + key,
}
text = "안녕하세요. 아오 왜 저장이 안될까"

# 입력한 텍스트를 변환하여 api를 호출

data = f"<speak>{text}</speak>".encode('utf-8').decode('latin1')
res = requests.post(url, headers = headers, data = data)

# 호출하여 불러온 결과값을 저장
with open('hellokakao.mp3', 'wb') as f:
    f.write(res.content)

PermissionError: [Errno 13] Permission denied: 'hellokakao.mp3'

In [439]:
import pygame, wave

pygame.init()
pygame.mixer.music.load("hellokakao.mp3")
pygame.mixer.music.play()
pygame.event.wait()

<Event(4352-AudioDeviceAdded {'which': 1, 'iscapture': 0})>

In [18]:
import pyaudio

po = pyaudio.PyAudio()

for index in range(po.get_device_count()): 

    desc = po.get_device_info_by_index(index)

    #if desc["name"] == "record":

    print ("DEVICE: %s  INDEX:  %s  RATE:  %s " %(desc["name"], index,  int(desc["defaultSampleRate"])))

DEVICE: Microsoft Sound Mapper - Input  INDEX:  0  RATE:  44100 
DEVICE: ¸¶ÀÌÅ©(Synaptics SmartAudio HD)  INDEX:  1  RATE:  44100 
DEVICE: Microsoft Sound Mapper - Output  INDEX:  2  RATE:  44100 
DEVICE: ½ºÇÇÄ¿(Synaptics SmartAudio HD)  INDEX:  3  RATE:  44100 
DEVICE: S23C340(ÀÎÅÚ(R) µð½ºÇÃ·¹ÀÌ ¿Àµð  INDEX:  4  RATE:  44100 


In [55]:
list(train_df["word"])

[727,
 727,
 727,
 167747,
 146464,
 146464,
 146464,
 169402,
 136476,
 121467,
 168987,
 168987,
 741,
 741,
 741,
 168296,
 169286,
 160615,
 156915,
 156915,
 156915,
 156915,
 168802,
 163506,
 163506,
 163506,
 120517,
 151508,
 151508,
 162721,
 166632,
 168939,
 168939,
 168939,
 169397,
 150806,
 150806,
 150806,
 169402,
 136476,
 165549,
 774,
 774,
 774,
 121467,
 159467,
 159467,
 159467,
 159467,
 168889,
 168889,
 168889,
 152296,
 785,
 133902,
 133902,
 164572,
 164572,
 164572,
 164572,
 164572,
 169224,
 161307,
 166830,
 166830,
 166830,
 152296,
 169149,
 131562,
 804,
 169159,
 168889,
 168889,
 152296,
 154674,
 154674,
 814,
 814,
 814,
 164423,
 164423,
 164423,
 164423,
 169332,
 164777,
 152415,
 825,
 825,
 169150,
 168889,
 168889,
 168890,
 161223,
 161223,
 162737,
 168889,
 168890,
 153767,
 837,
 151219,
 151219,
 151219,
 150713,
 43386,
 166302,
 166302,
 168004,
 168004,
 167306,
 167306,
 167306,
 167306,
 167306,
 168889,
 168889,
 168889,
 162611,

In [52]:
import numpy as np
import tensorflow as tf

# len(sentence) = 13
sentence=np.array(list(train_df["word"]))

W=tf.Variable(
    tf.random.uniform([169413,1000], -1.0, 1.0) ,name="W")

result=tf.nn.embedding_lookup(W,sentence)

In [61]:
result.shape

TensorShape([40793, 1000])